In [1]:
import torch
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
import time
import copy
import matplotlib.pyplot as plt
import os
import random

seed=42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Normalisation des images pour les modèles pré-entraînés PyTorch

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [5]:
image_directory = "/content/gdrive/MyDrive/ProjetFromage/data_fromage"
dataset_full = datasets.ImageFolder(image_directory, data_transforms)

# split en train, val et test à partir de la liste complète
np.random.seed(42)
samples_train, samples_test = train_test_split(dataset_full.samples)
samples_train, samples_val = train_test_split(samples_train,test_size=0.2)

print("Nombre d'images de train : %i" % len(samples_train))
print("Nombre d'images de val : %i" % len(samples_val))
print("Nombre d'images de test : %i" % len(samples_test))

Nombre d'images de train : 2288
Nombre d'images de val : 572
Nombre d'images de test : 954


In [ ]:
# on définit les datasets et loaders pytorch à partir des listes d'images de train / val / test
dataset_train = datasets.ImageFolder(image_directory, data_transforms)
dataset_train.samples = samples_train
dataset_train.imgs = samples_train

dataset_val = datasets.ImageFolder(image_directory, data_transforms)
dataset_val.samples = samples_val
dataset_val.imgs = samples_val

dataset_test = datasets.ImageFolder(image_directory, data_transforms)
dataset_test.samples = samples_test
dataset_test.imgs = samples_test

torch.manual_seed(42)

In [ ]:
# détermination du nombre de classes
# vérification que les labels sont bien dans [0, nb_classes]
labels=[x[1] for x in samples_train]
if np.min(labels) != 0:
    print("Error: labels should start at 0 (min is %i)" % np.min(labels))
    sys.exit(-1)
if np.max(labels) != (len(np.unique(labels))-1):
    print("Error: labels should go from 0 to Nclasses (max label = {}; Nclasse = {})".format(np.max(labels),len(np.unique(labels)))  )
    sys.exit(-1)
nb_classes = np.max(labels)+1
print("Apprentissage sur {} classes".format(nb_classes))

Apprentissage sur 43 classes


In [ ]:
# on définit le device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [ ]:
# on définit une fonction d'évaluation
def evaluate(model, dataset):
    avg_loss = 0.
    avg_accuracy = 0
    loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=False, num_workers=2)
    for data in loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        n_correct = torch.sum(preds == labels)
        
        avg_loss += loss.item()
        avg_accuracy += n_correct
        
    return avg_loss / len(dataset), float(avg_accuracy) / len(dataset)

# fonction d'entraînement du modèle
PRINT_LOSS = False

def train_model(model, loader_train, data_val, optimizer, criterion, n_epochs=10):
  
    since = time.time()
    best_acc=0
    best_model_wts = copy.deepcopy(model.state_dict())
    saved_acc=[]

    for epoch in range(n_epochs): 
        batch=0
        print("EPOCH % i" % epoch)
        for i, data in enumerate(loader_train):
            batch+=1
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device) 
            optimizer.zero_grad() 
            outputs = model(inputs) 
            
            loss = criterion(outputs, labels) 
            if PRINT_LOSS:
                model.train(False)
                loss_val, accuracy = evaluate(model, data_val)
                model.train(True)
                print("{} loss train: {:1.4f}\t val {:1.4f}\tAcc (val): {:.1%}".format(i, loss.item(), loss_val, accuracy   ))
            
            loss.backward() 
            optimizer.step() 
            #print(f'Batch {batch} : done')

        model.train(False)
        loss_val, accuracy = evaluate(model, data_val)
        saved_acc.append(accuracy)

        #early stopping
        if accuracy>best_acc:
          best_acc=accuracy
          best_model_wts = copy.deepcopy(model.state_dict())
        model.train(True)
        print("{} loss train: {:1.4f}\t val {:1.4f}\tAcc (val): {:.1%}".format(i, loss.item(), loss_val, accuracy   ))

    model.load_state_dict(best_model_wts)
    return best_acc


In [ ]:
#===== Transfer learning "simple" (sans fine tuning) =====

global_best_acc=0
best_param={}
times={}
best_net=models.mobilenet_v2(pretrained=True)

for batch_size in [8,16,32]:
  
  my_net = models.mobilenet_v2(pretrained=True)

  loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=2)

  for param in my_net.parameters():
      param.requires_grad = False

  my_net.classifier[1] = nn.Linear(in_features=my_net.classifier[1].in_features, out_features=nb_classes, bias=True)
  my_net.to(device) 
  my_net.train(True) 

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(my_net.classifier[1].parameters(), lr=0.001, momentum=0.9)

  print(f'Apprentissage en transfer learning : Batch size = {batch_size}')
  start=time.time()
  my_net.train(True)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  best_acc=train_model(my_net, loader_train, dataset_val, optimizer, criterion, n_epochs=10)
  times[batch_size]=time.time()-start
  print(f'Best Acc Val = {best_acc}\n')
  if best_acc>global_best_acc:
    global_best_acc=best_acc
    best_param={"Batch size":batch_size}
    best_net=my_net

print(">>> Best param :")
for key, item in best_param.items():
  print(f'{key} = {item}')

# évaluation
best_net.train(False)
loss, accuracy = evaluate(best_net, dataset_test)
print("Accuracy (test): %.1f%%" % (100 * accuracy))

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

Apprentissage en transfer learning : Batch size = 8
EPOCH  0
285 loss train: 2.4073	 val 0.1636	Acc (val): 40.7%
EPOCH  1
285 loss train: 2.5032	 val 0.1296	Acc (val): 53.1%
EPOCH  2
285 loss train: 1.3663	 val 0.1137	Acc (val): 55.9%
EPOCH  3
285 loss train: 1.6450	 val 0.1017	Acc (val): 59.8%
EPOCH  4
285 loss train: 1.8852	 val 0.0947	Acc (val): 62.1%
EPOCH  5
285 loss train: 1.3388	 val 0.0917	Acc (val): 63.5%
EPOCH  6
285 loss train: 1.3230	 val 0.0888	Acc (val): 63.3%
EPOCH  7
285 loss train: 1.0280	 val 0.0839	Acc (val): 64.7%
EPOCH  8
285 loss train: 0.9507	 val 0.0830	Acc (val): 64.3%
EPOCH  9
285 loss train: 0.9717	 val 0.0859	Acc (val): 63.6%
Best Acc Val = 0.6468531468531469

Apprentissage en transfer learning : Batch size = 16
EPOCH  0
142 loss train: 2.9328	 val 0.1873	Acc (val): 33.2%
EPOCH  1
142 loss train: 2.7520	 val 0.1548	Acc (val): 47.6%
EPOCH  2
142 loss train: 1.8327	 val 0.1352	Acc (val): 53.7%
EPOCH  3
142 loss train: 1.5649	 val 0.1216	Acc (val): 57.0%
EPOCH 

In [ ]:
torch.cuda.empty_cache()

#===== Fine tuning =====

global_best_acc=0
best_param={}
times_ft={}
best_net_ft=models.mobilenet_v2(pretrained=True)
for batch_size in [8,16,32]:
  # on réinitialise MobileNet
  my_net_ft = models.mobilenet_v2(pretrained=True)
  my_net_ft.classifier[1] = nn.Linear(in_features=my_net_ft.classifier[1].in_features, out_features=nb_classes, bias=True)
  my_net_ft.to(device)

  loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=2)

  # cette fois on veut updater tous les paramètres
  params_to_update = my_net_ft.parameters()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

  # on ré-entraîne
  print(f'Apprentissage avec fine tuning : Batch size = {batch_size}')
  start=time.time()
  my_net_ft.train(True)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  best_acc=train_model(my_net_ft, loader_train, dataset_val, optimizer, criterion, n_epochs=10)
  times_ft[batch_size]=time.time()-start
  print(f'Best Acc Val = {best_acc}\n')
  if best_acc>global_best_acc:
    global_best_acc=best_acc
    best_param={"Batch size":batch_size}
    best_net_ft=my_net_ft

print(">>> Best param :")
for key, item in best_param.items():
  print(f'{key} = {item}')

# on ré-évalue les performances
best_net_ft.train(False)
loss, accuracy = evaluate(best_net_ft, dataset_test)
print("Accuracy (test): %.1f%%" % (100 * accuracy))

Apprentissage avec fine tuning : Batch size = 8
EPOCH  0
285 loss train: 1.2559	 val 0.1008	Acc (val): 56.1%
EPOCH  1
285 loss train: 1.8478	 val 0.0740	Acc (val): 68.9%
EPOCH  2
285 loss train: 0.4219	 val 0.0642	Acc (val): 72.2%
EPOCH  3
285 loss train: 0.8089	 val 0.0557	Acc (val): 76.0%
EPOCH  4
285 loss train: 0.3836	 val 0.0674	Acc (val): 72.7%
EPOCH  5
285 loss train: 0.2616	 val 0.0531	Acc (val): 79.0%
EPOCH  6
285 loss train: 0.0384	 val 0.0559	Acc (val): 77.8%
EPOCH  7
285 loss train: 0.0275	 val 0.0560	Acc (val): 78.0%
EPOCH  8
285 loss train: 0.0608	 val 0.0595	Acc (val): 77.1%
EPOCH  9
285 loss train: 0.0408	 val 0.0601	Acc (val): 77.6%
Best Acc Val = 0.7902097902097902

Apprentissage avec fine tuning : Batch size = 16
EPOCH  0
142 loss train: 2.0029	 val 0.1263	Acc (val): 54.7%
EPOCH  1
142 loss train: 1.7537	 val 0.0847	Acc (val): 64.5%
EPOCH  2
142 loss train: 0.7962	 val 0.0695	Acc (val): 69.4%
EPOCH  3
142 loss train: 0.7964	 val 0.0584	Acc (val): 75.7%
EPOCH  4
142 l

In [ ]:
times

{8: 1190.2223868370056, 16: 677.8826870918274, 32: 667.2306008338928}

In [ ]:
times_ft

{8: 738.8128876686096, 16: 684.4567852020264, 32: 690.8387215137482}